In [ ]:
!del /f /q C:\Users\DELL\Desktop\litmus_app\litmus_main.py
!mkdir C:\Users\DELL\Desktop\litmus_app


In [2]:
%%writefile C:\Users\DELL\Desktop\litmus_app\litmus_main.py
# === PART 1: STREAMLIT SETUP + RUN BACKTEST MODE ===
import streamlit as st
import yfinance as yf
import backtrader as bt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime, importlib
from pathlib import Path

# === Helper Functions ===
def clean_data(data: pd.DataFrame):
    df = data.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df.columns = [c.lower().strip() for c in df.columns]
    df.dropna(inplace=True)
    needed = ["open", "high", "low", "close", "volume"]
    for c in needed:
        if c not in df.columns:
            raise ValueError(f"Missing required column: {c}")
    return df

def ensure_folder():
    folder = Path("Backtests_History")
    folder.mkdir(exist_ok=True)
    return folder

def save_backtest_results(df: pd.DataFrame, fig):
    folder = ensure_folder()
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_path = folder / f"results_{ts}.csv"
    png_path = folder / f"chart_{ts}.png"
    df.to_csv(csv_path)
    if importlib.util.find_spec("kaleido") is not None:
        try:
            fig.write_image(png_path)
        except Exception as e:
            print("⚠️ Kaleido error:", e)
    return csv_path, png_path

def compute_equity_curve(data: pd.DataFrame, cash: float):
    df = data.copy()
    df["returns"] = df["close"].pct_change().fillna(0)
    df["equity_curve"] = (1 + df["returns"]).cumprod() * cash
    df["profit_loss"] = df["equity_curve"] - cash
    return df

# === Sample Strategy: SMA Crossover ===
class SmaCross(bt.Strategy):
    params = dict(pfast=10, pslow=30)
    def __init__(self):
        self.sma1 = bt.ind.SMA(period=self.p.pfast)
        self.sma2 = bt.ind.SMA(period=self.p.pslow)
        self.cross = bt.ind.CrossOver(self.sma1, self.sma2)
        self.signals = []
    def next(self):
        if not self.position and self.cross > 0:
            self.buy()
            self.signals.append((self.data.datetime.date(0), "BUY", self.data.close[0]))
        elif self.position and self.cross < 0:
            self.sell()
            self.signals.append((self.data.datetime.date(0), "SELL", self.data.close[0]))

STRATS = {"SMA Crossover": SmaCross}

def run_bt(data, strategy_class, cash=10000):
    df = clean_data(data)
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)
    feed = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(feed)
    cerebro.addstrategy(strategy_class)
    results = cerebro.run()
    strat = results[0]
    signals = strat.signals
    start_val = cash
    end_val = cerebro.broker.getvalue()
    ret_pct = round((end_val / start_val - 1) * 100, 2)
    eq_data = compute_equity_curve(df, cash)
    return ret_pct, end_val, signals, eq_data

def plot_price_and_equity(df_price, eq_data, signals, title):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.6, 0.4])
    fig.add_trace(go.Candlestick(
        x=df_price.index,
        open=df_price["open"],
        high=df_price["high"],
        low=df_price["low"],
        close=df_price["close"],
        name="Price",
        increasing_line_color="#00ff99",
        decreasing_line_color="#ff4d4d"
    ), row=1, col=1)
    for dt, act, price in signals:
        fig.add_trace(go.Scatter(
            x=[dt], y=[price],
            mode="markers",
            marker_symbol="triangle-up" if act == "BUY" else "triangle-down",
            marker_color="lime" if act == "BUY" else "red",
            name=act
        ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=eq_data.index,
        y=eq_data["equity_curve"],
        mode="lines",
        name="Equity Curve",
        line=dict(color="cyan", width=3)
    ), row=2, col=1)
    fig.update_layout(template="plotly_dark", height=900, title=title)
    return fig

# === STREAMLIT UI ===
st.set_page_config(page_title="Litmus Quant Dashboard", layout="wide")
st.title("🧪 Litmus Quant Dashboard")
st.sidebar.title("Select Mode")

mode = st.sidebar.radio(
    "Choose Mode",
    [
        "🚀 Run Backtest",
        "📚 Backtest Dashboard",
        "🧪 Strategy Lab",
        "🌍 Multi-Asset Comparison",
        "📈 Advanced Analytics"
    ]
)


if mode == "🚀 Run Backtest":
    st.sidebar.header("Run Backtest Settings")
    sym = st.sidebar.text_input("Symbol", "AAPL")
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", min_value=1000.0, value=10000.0)
    run = st.sidebar.button("Run Backtest")

    if run:
        data = yf.download(sym, start=start, end=end, progress=False)
        data = clean_data(data)
        st.subheader(f"📈 Backtest Results for {sym}")

        r, ev, sigs, eq = run_bt(data, STRATS["SMA Crossover"], cash)
        st.success(f"✅ Return: {r}% | Final Value: ${ev:,.2f}")

        fig = plot_price_and_equity(data, eq, sigs, title=f"SMA Crossover — {sym}")
        st.plotly_chart(fig, use_container_width=True)

        # Save button
        if st.button("💾 Save Results as CSV & PNG"):
            csv_path, png_path = save_backtest_results(eq, fig)
            st.success(f"Results saved!\n📁 CSV: {csv_path}\n🖼️ PNG: {png_path}")


Writing C:\Users\DELL\Desktop\litmus_app\litmus_main.py


In [3]:
%%writefile -a C:\Users\DELL\Desktop\litmus_app\litmus_main.py
# PART 2 — BASE STRATEGY + STRATEGY DEFINITIONS

class BaseStrategy(bt.Strategy):
    def __init__(self):
        self.signals = []   # (date, BUY/SELL, price)
        self.trades = []    # closed trade PnL

    def log_signal(self, action):
        dt = self.data.datetime.date(0)
        price = self.data.close[0]
        self.signals.append((dt, action, price))

    def notify_trade(self, trade):
        if trade.isclosed:
            self.trades.append(trade.pnl)

# 1. SMA Crossover
class SmaCross(BaseStrategy):
    params = dict(pfast=10, pslow=30)
    def __init__(self):
        super().__init__()
        sma1 = bt.ind.SMA(period=self.p.pfast)
        sma2 = bt.ind.SMA(period=self.p.pslow)
        self.cross = bt.ind.CrossOver(sma1, sma2)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy()
            self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell()
            self.log_signal("SELL")

# 2. RSI Mean Reversion
class Rsi(BaseStrategy):
    params = dict(period=14, low=30, high=70)
    def __init__(self):
        super().__init__()
        self.rsi = bt.ind.RSI(period=self.p.period)
    def next(self):
        if not self.position and self.rsi < self.p.low:
            self.buy()
            self.log_signal("BUY")
        elif self.position and self.rsi > self.p.high:
            self.sell()
            self.log_signal("SELL")

# 3. Bollinger Bands Reversion
class Bollinger(BaseStrategy):
    params = dict(period=20, dev=2)
    def __init__(self):
        super().__init__()
        self.bb = bt.ind.BollingerBands(period=self.p.period, devfactor=self.p.dev)
    def next(self):
        if not self.position and self.data.close < self.bb.bot:
            self.buy()
            self.log_signal("BUY")
        elif self.position and self.data.close > self.bb.top:
            self.sell()
            self.log_signal("SELL")

# 4. Momentum Breakout
class Momentum(BaseStrategy):
    params = dict(period=20)
    def next(self):
        if len(self.data) <= self.p.period:
            return
        if not self.position and self.data.close[0] > self.data.close[-self.p.period]:
            self.buy()
            self.log_signal("BUY")
        elif self.position and self.data.close[0] < self.data.close[-self.p.period]:
            self.sell()
            self.log_signal("SELL")

# 5. MACD Trend Following
class MACD(BaseStrategy):
    params = dict(fast=12, slow=26, sig=9)
    def __init__(self):
        super().__init__()
        m = bt.ind.MACD(period_me1=self.p.fast, period_me2=self.p.slow, period_signal=self.p.sig)
        self.cross = bt.ind.CrossOver(m.macd, m.signal)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy()
            self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell()
            self.log_signal("SELL")

# 6. Buy & Hold
class BuyHold(BaseStrategy):
    def next(self):
        if not self.position:
            self.buy()
            self.log_signal("BUY")

# Placeholder strategies (optional)
class EMA(BaseStrategy): pass
class DualSMA(BaseStrategy): pass
class Donchian(BaseStrategy): pass
class Stoch(BaseStrategy): pass
class ADX(BaseStrategy): pass
class VWAP(BaseStrategy): pass
class ATRStop(BaseStrategy): pass
class Seasonal(BaseStrategy): pass
class RiskParity(BaseStrategy): pass
class Factor(BaseStrategy): pass
class Breakout(BaseStrategy): pass

# Register all strategies
STRATS = {
    "SMA Crossover": SmaCross,
    "RSI": Rsi,
    "Bollinger": Bollinger,
    "Momentum": Momentum,
    "MACD": MACD,
    "Buy & Hold": BuyHold,
    "EMA": EMA,
    "Dual SMA": DualSMA,
    "Donchian": Donchian,
    "Stoch": Stoch,
    "ADX": ADX,
    "VWAP": VWAP,
    "ATR Stop": ATRStop,
    "Seasonal": Seasonal,
    "Risk Parity": RiskParity,
    "Factor": Factor,
    "Breakout": Breakout,
}




Appending to C:\Users\DELL\Desktop\litmus_app\litmus_main.py


In [4]:
%%writefile -a C:\Users\DELL\Desktop\litmus_app\litmus_main.py
# PART 3 — BACKTEST ENGINE

def run_bt(data: pd.DataFrame, strategy_class, cash=10000, **kwargs):
    """
    Runs a single strategy backtest on a single asset.
    Returns:
      - return %
      - final value
      - signals list
      - win rate %
      - max drawdown %
      - equity curve DataFrame
    """
    # Clean and prepare data
    df = clean_data(data.copy())

    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)

    # Load data feed
    feed = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(feed)

    # Add chosen strategy
    cerebro.addstrategy(strategy_class, **kwargs)

    # Run backtest
    results = cerebro.run()
    strat = results[0] if results else None

    # Extract signals + trades
    signals = getattr(strat, "signals", [])
    trades = getattr(strat, "trades", [])

    # Compute return %
    start_val = cash
    end_val = cerebro.broker.getvalue()
    ret_pct = round((end_val / start_val - 1) * 100, 2)

    # Compute win rate %
    win_rate = round(
        100 * len([t for t in trades if t > 0]) / len(trades),
        2
    ) if trades else 0

    # Compute equity curve (price-based baseline)
    eq_data = compute_equity_curve(df, cash)

    # Compute max drawdown %
    rolling_max = eq_data["equity_curve"].cummax()
    drawdown_series = (eq_data["equity_curve"] - rolling_max) / rolling_max
    max_dd = round(drawdown_series.min() * 100, 2)

    return (
        ret_pct,        # % return
        end_val,        # final value
        signals,        # buy/sell signal list
        win_rate,       # win rate %
        max_dd,         # max drawdown %
        eq_data         # equity curve dataframe
    )




Appending to C:\Users\DELL\Desktop\litmus_app\litmus_main.py


In [5]:
%%writefile -a C:\Users\DELL\Desktop\litmus_app\litmus_main.py
# PART 4 — PRICE + EQUITY CURVE CHART

def plot_price_and_equity(df_price, eq_data, signals, title="Price & Strategy Equity Curve"):
    """Draws 2-panel chart:
       (1) Candlestick price chart with buy/sell markers
       (2) Strategy equity curve below
    """

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        row_heights=[0.65, 0.35],
        subplot_titles=("Price Chart", "Equity Curve")
    )

    # PRICE (Candlestick)
    fig.add_trace(go.Candlestick(
        x=df_price.index,
        open=df_price["open"],
        high=df_price["high"],
        low=df_price["low"],
        close=df_price["close"],
        name="Price",
        increasing_line_color="#00ff99",
        decreasing_line_color="#ff4d4d",
        increasing_fillcolor="rgba(0,255,153,0.4)",
        decreasing_fillcolor="rgba(255,77,77,0.4)",
        whiskerwidth=0.6
    ), row=1, col=1)

    # BUY/SELL MARKERS
    for dt, action, price in signals:
        fig.add_trace(go.Scatter(
            x=[dt], y=[price],
            mode="markers",
            marker_symbol="triangle-up" if action == "BUY" else "triangle-down",
            marker_color="lime" if action == "BUY" else "red",
            marker_size=12,
            name=action,
            hovertemplate=f"{action} @ {price:.2f}<extra></extra>"
        ), row=1, col=1)

    # EQUITY CURVE
    fig.add_trace(go.Scatter(
        x=eq_data.index,
        y=eq_data["equity_curve"],
        mode="lines",
        name="Equity Curve",
        line=dict(width=3, color="cyan"),
        hovertemplate="Equity: %{y:.2f}<extra></extra>"
    ), row=2, col=1)

    # LAYOUT
    fig.update_layout(
        template="plotly_dark",
        hovermode="x unified",
        dragmode="zoom",
        title=title,
        height=900,
        plot_bgcolor="rgba(13,13,13,1)",
        paper_bgcolor="rgba(13,13,13,1)",
        font=dict(color="white", family="Arial"),
        xaxis_rangeslider_visible=False,
    )

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)

    return fig




Appending to C:\Users\DELL\Desktop\litmus_app\litmus_main.py


In [6]:
%%writefile -a C:\Users\DELL\Desktop\litmus_app\litmus_main.py

# PART — MODE 2: BACKTEST DASHBOARD
if mode == "📚 Backtest Dashboard":
    st.sidebar.header("Backtest Dashboard Settings")

    history_folder = Path("Backtests_History")
    if not history_folder.exists():
        st.warning("⚠️ No Backtest History Found. Run a backtest first.")
    else:
        files = sorted(history_folder.glob("results_*.csv"), reverse=True)
        if not files:
            st.info("No saved backtest results yet. Run one to generate files.")
        else:
            st.subheader("📚 Saved Backtest Results")
            file_names = [f.name for f in files]
            selected_file = st.selectbox("Select a saved backtest to view:", file_names)

            if selected_file:
                file_path = history_folder / selected_file
                df = pd.read_csv(file_path)
                st.success(f"✅ Loaded: {selected_file}")

                # Display data
                st.dataframe(df)

                # Compute summary stats if present
                if {"Return %", "Win Rate %", "Max DD %", "Final Value"}.issubset(df.columns):
                    st.markdown("### 📊 Summary Statistics")
                    col1, col2, col3, col4 = st.columns(4)
                    col1.metric("Return %", f"{df['Return %'].mean():.2f}%")
                    col2.metric("Win Rate %", f"{df['Win Rate %'].mean():.2f}%")
                    col3.metric("Max DD %", f"{df['Max DD %'].mean():.2f}%")
                    col4.metric("Final Value", f"${df['Final Value'].mean():,.2f}")

                # Plot results if columns exist
                if "equity_curve" in df.columns and "close" in df.columns:
                    st.markdown("### 📈 Equity Curve")
                    fig = go.Figure()
                    fig.add_trace(go.Scatter(
                        x=df.index,
                        y=df["equity_curve"],
                        mode="lines",
                        name="Equity Curve",
                        line=dict(color="cyan", width=3)
                    ))
                    fig.update_layout(
                        template="plotly_dark",
                        height=600,
                        title="📈 Saved Equity Curve",
                        plot_bgcolor="rgba(13,13,13,1)",
                        paper_bgcolor="rgba(13,13,13,1)",
                        font=dict(color="white")
                    )
                    st.plotly_chart(fig, use_container_width=True)
                    # 💾 Manual Save Button for Backtest Results
                    save_btn = st.button("💾 Save Results as CSV & PNG")
                    
                    if save_btn:
                        try:
                            csv_path, png_path = save_backtest_results(df, fig)
                            st.success("✅ Backtest results saved successfully!")
                            st.caption(f"📁 CSV saved at: {csv_path}")
                            st.caption(f"🖼️ PNG saved at: {png_path}")
                        except Exception as e:
                            st.error(f"⚠️ Could not save results: {e}")

                # Option to download again
                st.download_button(
                    label="⬇️ Download CSV Copy",
                    data=df.to_csv(index=False),
                    file_name=selected_file,
                    mime="text/csv"
                )


Appending to C:\Users\DELL\Desktop\litmus_app\litmus_main.py


In [7]:
%%writefile -a C:\Users\DELL\Desktop\litmus_app\litmus_main.py
# PART 6 — MODE 2: STRATEGY LAB (TUNE & TEST)

if mode == "🧪 Strategy Lab":
    st.sidebar.header("Strategy Lab Settings")

    sym = st.sidebar.text_input("Symbol (Ticker)", "AAPL")
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", min_value=1000.0, value=10000.0)

    # Choose a single strategy to tune
    strat_choice = st.sidebar.selectbox("Choose Strategy", list(STRATS.keys()))

    kwargs = {}

    # Add parameter sliders based on strategy type
    if strat_choice == "SMA Crossover":
        kwargs["pfast"] = st.sidebar.slider("Fast SMA Period", 5, 50, 10)
        kwargs["pslow"] = st.sidebar.slider("Slow SMA Period", 20, 200, 30)
    elif strat_choice == "RSI":
        kwargs["period"] = st.sidebar.slider("RSI Period", 5, 30, 14)
        kwargs["low"] = st.sidebar.slider("Buy Threshold", 10, 40, 30)
        kwargs["high"] = st.sidebar.slider("Sell Threshold", 60, 90, 70)
    elif strat_choice == "Bollinger":
        kwargs["period"] = st.sidebar.slider("Period", 10, 40, 20)
        kwargs["dev"] = st.sidebar.slider("Standard Deviation", 1, 3, 2)

    run = st.sidebar.button("Run Strategy Lab")

    if run:
        data = yf.download(sym, start=start, end=end, progress=False)
        data = clean_data(data)
        st.subheader(f"🔬 Strategy Lab — {strat_choice} on {sym}")

        r, ev, sigs, wr, dd, eq = run_bt(data, STRATS[strat_choice], cash, **kwargs)

        # Display performance summary
        st.success(
            f"""
            **Performance Summary**
            - Return: `{r}%`
            - Final Value: `${ev:,.2f}`
            - Win Rate: `{wr}%`
            - Max Drawdown: `{dd}%`
            """
        )

        # Show price + equity curve chart
        fig = plot_price_and_equity(data, eq, sigs, title=f"{strat_choice} — {sym}")
        st.plotly_chart(fig, use_container_width=True)


Appending to C:\Users\DELL\Desktop\litmus_app\litmus_main.py


In [8]:
%%writefile -a C:\Users\DELL\Desktop\litmus_app\litmus_main.py
# PART 7 — MODE 3: MULTI-ASSET COMPARISON

if mode == "🌍 Multi-Asset Comparison":
    st.sidebar.header("Multi-Asset Comparison Settings")

    tickers = st.sidebar.text_input("Enter tickers (comma-separated)", "AAPL, MSFT, TSLA, NVDA")
    strat_choice = st.sidebar.selectbox("Choose Strategy", list(STRATS.keys()))
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", min_value=1000.0, value=10000.0)

    run = st.sidebar.button("Run Multi-Asset Backtest")

    if run:
        tickers_list = [t.strip().upper() for t in tickers.split(",")]
        results = []
        combined_fig = go.Figure()

        for sym in tickers_list:
            data = yf.download(sym, start=start, end=end, progress=False)
            data = clean_data(data)
            r, ev, sigs, wr, dd, eq = run_bt(data, STRATS[strat_choice], cash)

            results.append({
                "Symbol": sym,
                "Return %": r,
                "Final Value": f"${ev:,.2f}",
                "Win Rate %": wr,
                "Max DD %": dd
            })

            combined_fig.add_trace(go.Scatter(
                x=eq.index,
                y=eq["equity_curve"],
                mode="lines",
                name=sym
            ))

        df = pd.DataFrame(results).set_index("Symbol")
        st.subheader(f"🌍 Multi-Asset Backtest — {strat_choice}")
        st.dataframe(df)

        combined_fig.update_layout(
            title=f"Price Comparison ({strat_choice})",
            template="plotly_dark",
            hovermode="x unified",
            height=700,
            plot_bgcolor="rgba(13,13,13,1)",
            paper_bgcolor="rgba(13,13,13,1)",
            font=dict(color="white")
        )

        st.plotly_chart(combined_fig, use_container_width=True)


Appending to C:\Users\DELL\Desktop\litmus_app\litmus_main.py


In [9]:
%%writefile -a C:\Users\DELL\Desktop\litmus_app\litmus_main.py
# PART 8 — MODE 4: ADVANCED ANALYTICS DASHBOARD

if mode == "📈 Advanced Analytics":
    st.sidebar.header("Advanced Analytics Settings")

    tickers = st.sidebar.text_input("Enter tickers (comma-separated)", "AAPL, MSFT, TSLA, NVDA")
    strat_choice = st.sidebar.selectbox("Choose Strategy", list(STRATS.keys()))
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", min_value=1000.0, value=10000.0)

    run = st.sidebar.button("Run Advanced Analytics")

    if run:
        tickers_list = [t.strip().upper() for t in tickers.split(",")]
        results = []
        portfolio_equity = pd.DataFrame()

        for sym in tickers_list:
            data = yf.download(sym, start=start, end=end, progress=False)
            data = clean_data(data)
            r, ev, sigs, wr, dd, eq = run_bt(data, STRATS[strat_choice], cash)

            # Compute analytics
            daily_ret = eq["equity_curve"].pct_change().dropna()
            sharpe = round((daily_ret.mean() / daily_ret.std()) * np.sqrt(252), 2)
            vol = round(daily_ret.std() * 100, 2)
            profit_factor = round((daily_ret[daily_ret > 0].sum() / abs(daily_ret[daily_ret < 0].sum())), 2) if (daily_ret < 0).any() else 0

            results.append({
                "Symbol": sym,
                "Return %": r,
                "Final Value": f"${ev:,.2f}",
                "Sharpe Ratio": sharpe,
                "Volatility %": vol,
                "Win Rate %": wr,
                "Profit Factor": profit_factor,
                "Max DD %": dd
            })

            portfolio_equity[sym] = eq["equity_curve"]

        df = pd.DataFrame(results).set_index("Symbol")

        st.subheader(f"📊 Advanced Performance Table — {strat_choice}")
        st.dataframe(df)

                # Portfolio metrics
        total_init = len(tickers_list) * cash
        total_final = portfolio_equity.iloc[-1].sum()
        portfolio_return = round((total_final / total_init - 1) * 100, 2)
        portfolio_sharpe = round(((portfolio_equity.pct_change().mean().mean()) /
                                  (portfolio_equity.pct_change().std().mean())) * np.sqrt(252), 2)
        max_drawdown = round((((portfolio_equity.sum() / len(tickers_list)) /
                               (portfolio_equity.sum() / len(tickers_list)).cummax() - 1).min()) * 100, 2)

        # Portfolio Summary (horizontal layout)
        st.markdown("### 💼 Portfolio Summary")
        
        col1, col2, col3, col4, col5 = st.columns(5)
        
        with col1:
            st.metric("💼 Total Initial", f"${total_init:,.2f}")
        
        with col2:
            st.metric("💰 Total Final Value", f"${total_final:,.2f}")
        
        with col3:
            st.metric("📈 Portfolio Return %", f"{portfolio_return}%")
        
        with col4:
            st.metric("⚖️ Portfolio Sharpe", portfolio_sharpe)
        
        with col5:
            st.metric("📉 Max Drawdown %", f"{max_drawdown}")
        
        # Plot combined portfolio equity curve
        combined_fig = go.Figure()
        for sym in portfolio_equity.columns:
            combined_fig.add_trace(go.Scatter(
                x=portfolio_equity.index,
                y=portfolio_equity[sym],
                mode="lines",
                name=sym
            ))
        
        combined_fig.update_layout(
            title=f"📈 Portfolio Equity Curves ({strat_choice})",
            template="plotly_dark",
            hovermode="x unified",
            height=700,
            plot_bgcolor="rgba(13,13,13,1)",
            paper_bgcolor="rgba(13,13,13,1)",
            font=dict(color="white")
        )
        
        st.plotly_chart(combined_fig, use_container_width=True)
        
                   


Appending to C:\Users\DELL\Desktop\litmus_app\litmus_main.py


In [ ]:
!streamlit run C:\Users\DELL\Desktop\litmus_app\litmus_main.py
